In [1]:
import pandas as pd
import datetime

## Read Data

In [2]:
all_data = pd.read_csv("train.csv")

In [3]:
# summarize the shape of the dataset
all_data.shape

(387229, 136)

Split the DATE coulmn to 3 seperated columns named month, year and weekday

In [114]:
# from datetime import date
# import datetime

# df['Day'] = df['DATE'].apply(lambda x: 
#                       (date(datetime.datetime.strptime(x,"%Y-%m-%d").year,datetime.datetime.strptime(x,"%Y-%m-%d").month,
#                       datetime.datetime.strptime(x,"%Y-%m-%d").day)-date(2019,4,10)).days)
# df.drop(['DATE'],axis=1,inplace=True)

In [3]:
all_data['month'] = all_data['DATE'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").month)  
all_data['year'] = all_data['DATE'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").year)                                                                         
all_data['weekday'] = all_data['DATE'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").weekday())

In [4]:
#Remove date column
all_data = all_data.drop(columns='DATE')

In [5]:
# summarize the shape of the dataset
all_data.shape

(198126, 138)

In [ ]:
# summarize each variable
print(all_data.describe())

In [7]:
df = pd.DataFrame(all_data)

Get all mean, max and min values for the same numerical variables and do a function in order to have one indicatioe columns instead of 3 columns

In [12]:
numerical_dic = {'temperature_prev7d':3,'temperature_prev3d':6, 'rain_mm_prev7d': 9, 'rain_mm_prev3d': 12, 'humidity_prev7d' : 15,'humidity_prev3d':18, 'wind_speed_prev7d':21 , 'wind_speed_prev3d':24, 'wind_speed_prev7d' : 27, 'wind_speed_prev3d': 30, 'pressure_prev7d':33, 'pressure_prev3d': 36, 'temperature_f_next14d': 39, 'temperature_f_next7d': 42, 'rain_mm_f_next14d':45, 'rain_mm_f_next7d':48, 'humidity_f_next14d':51, 'humidity_f_next7d' : 54, 'speed_f_next14d':57, 'speed_f_next7d':60, 'pressure_f_next14d':63, 'pressure_f_next7d':66,       }
Numerical_df = df.loc[: , 'aircon_sum_wo_prev7d':'min_pressure_f_next7d']


In [13]:
def new_numerical_df(numerical_dic, Numerical_df):
  i = 0
  while(i<len(numerical_dic)):
    if list(numerical_dic.keys())[i] == 'aircon_sum_wo_prev':
      index=2
    else:
      index = 3
    
    df_new = Numerical_df[Numerical_df.columns[(list(numerical_dic.values())[i]):(list(numerical_dic.values())[i]+index)]]
    Numerical_df[list(numerical_dic.keys())[i]] = df_new.mean(axis=1)

    i= i+1


  # Numerical_df = Numerical_df.drop(columns = ['aircon_sum_wo_prev7d','aircon_sum_wo_prev14d'])
  Numerical_df=Numerical_df.drop(Numerical_df.columns.to_series()['mean_temperature_prev7d':'min_pressure_f_next7d'], axis=1)
  return Numerical_df


In [14]:
New_Numerical = new_numerical_df(numerical_dic, Numerical_df)

In [15]:
New_Numerical.head()

,aircon_sum_wo_prev7d,aircon_sum_wo_prev14d,aircon_sum_target_next14d,temperature_prev7d,temperature_prev3d,rain_mm_prev7d,rain_mm_prev3d,humidity_prev7d,humidity_prev3d,wind_speed_prev7d,wind_speed_prev3d,pressure_prev7d,pressure_prev3d,temperature_f_next14d,temperature_f_next7d,rain_mm_f_next14d,rain_mm_f_next7d,humidity_f_next14d,humidity_f_next7d,speed_f_next14d,speed_f_next7d,pressure_f_next14d,pressure_f_next7d
0,0.0,0.0,0,10.096667,11.666667,3.276667,3.966667,62.903333,69.890000,1013.666667,1008.000000,8.333333,6.713333,8.163333,8.736667,73.763333,77.333333,11.620000,13.620000,1019.236667,1015.046667,8.346667,45.586667
1,0.0,0.0,0,12.236667,12.666667,3.466667,4.356667,66.143333,75.666667,1011.286667,1007.000000,9.833333,6.286667,5.373333,5.776667,73.310000,77.286667,11.523333,13.430000,1020.903333,1019.570000,9.456667,48.722222
2,0.0,0.0,0,12.190000,12.666667,8.100000,9.290000,72.570000,78.776667,1010.236667,1007.443333,11.453333,7.430000,5.410000,6.103333,73.000000,76.810000,11.430000,13.523333,1022.046667,1021.190000,10.985556,53.545556
3,0.0,0.0,0,11.763333,11.666667,8.123333,9.376667,73.430000,79.776667,1009.476667,1009.890000,12.523333,7.620000,5.333333,5.990000,72.356667,75.713333,11.380000,13.046667,1020.620000,1022.430000,10.517778,54.194444
4,0.0,0.0,0,10.523333,7.110000,8.430000,9.410000,74.430000,85.333333,1010.620000,1012.556667,12.833333,8.286667,5.363333,5.976667,72.023333,74.953333,11.213333,11.236667,1019.430000,1023.000000,8.687778,56.391111


Split the origin DataFrame to subdataframes and sort the numerical datafram according to their time (prev7d,prev3d,next7d, next14d)

In [106]:
categorical_df = df.loc[: ,'CELL_TYPE_Macro':'CELL_TYPE_micro'] + df.loc[: ,'GEOGRAPHIC_CLUSTER_K_0':'GEOGRAPHIC_CLUSTER_K_9']
aircon_sum_wo_df = df.loc[:, 'aircon_sum_wo_prev7d' : 'aircon_sum_target_next14d']
N_TRANSPORTED_SITES_df = df.loc[:,'N_TRANSPORTED_SITES']
Numerical_df = df.loc[: , 'mean_temperature_prev7d':'min_pressure_f_next7d']
Numerical_df = Numerical_df.reindex(sorted(Numerical_df.columns, reverse=True), axis=1)
alarm_df = df.loc[: , 'equipment_sum_alarms_prev14d':'temperature_sum_alarms_prev3d']
alarm_df = alarm_df.reindex(sorted(alarm_df.columns), axis=1)
persistance_df = (df.loc[:, 'equipment_max_persistance_prev7d': 'temperature_min_persistance_prev3d'])
persistance_df = persistance_df.reindex(sorted(persistance_df.columns, reverse=True), axis=1)
skewKurt_df = df.loc[:,'skew_equipment_alarms_prev14d': 'kurt_temperature_alarms_prev14d']
Date_df = df.loc[:,'month':'weekday']

In [107]:
Numerical_df.head()

,min_wind_speed_prev7d,min_wind_speed_prev3d,min_wind_speed_f_next7d,min_wind_speed_f_next14d,min_temperature_prev7d,min_temperature_prev3d,min_temperature_f_next7d,min_temperature_f_next14d,min_rain_mm_prev7d,min_rain_mm_prev3d,min_rain_mm_f_next7d,min_rain_mm_f_next14d,min_pressure_prev7d,min_pressure_prev3d,min_pressure_f_next7d,min_pressure_f_next14d,min_humidity_prev7d,min_humidity_prev3d,min_humidity_f_next7d,min_humidity_f_next14d,mean_wind_speed_prev7d,mean_wind_speed_prev3d,mean_wind_speed_f_next7d,mean_wind_speed_f_next14d,mean_temperature_prev7d,mean_temperature_prev3d,mean_temperature_f_next7d,mean_temperature_f_next14d,mean_rain_mm_prev7d,mean_rain_mm_prev3d,mean_rain_mm_f_next7d,mean_rain_mm_f_next14d,mean_pressure_prev7d,mean_pressure_prev3d,mean_pressure_f_next7d,mean_pressure_f_next14d,mean_humidity_prev7d,mean_humidity_prev3d,mean_humidity_f_next7d,mean_humidity_f_next14d,max_wind_speed_prev7d,max_wind_speed_prev3d,max_wind_speed_f_next7d,max_wind_speed_f_next14d,max_temperature_prev7d,max_temperature_prev3d,max_temperature_f_next7d,max_temperature_f_next14d,max_rain_mm_prev7d,max_rain_mm_prev3d,max_rain_mm_f_next7d,max_rain_mm_f_next14d,max_pressure_prev7d,max_pressure_prev3d,max_pressure_f_next7d,max_pressure_f_next14d,max_humidity_prev7d,max_humidity_prev3d,max_humidity_f_next7d,max_humidity_f_next14d
0,5.0,5.0,10.0,5.0,6.0,9.0,5.0,5.0,0.0,0.3,0.2,0.1,1006.0,1006.0,1009.0,1009.0,45.0,58.0,63.0,58.0,11.43,8.00,13.86,12.86,10.29,12.00,6.14,8.00,1.33,3.10,6.41,4.79,1013.00,1008.00,1016.14,1020.71,62.71,70.67,80.00,74.29,16.0,12.0,17.0,17.0,14.0,14.0,9.0,12.0,8.5,8.5,19.6,19.6,1022.0,1010.0,1020.0,1028.0,81.0,81.0,89.0,89.0
1,5.0,5.0,10.0,5.0,9.0,9.0,5.0,5.0,0.0,0.3,0.2,0.1,1006.0,1006.0,1013.0,1013.0,51.0,71.0,63.0,58.0,11.57,8.00,13.29,12.57,11.71,13.00,5.86,8.50,1.90,4.27,4.63,3.52,1010.86,1007.00,1018.71,1021.71,66.43,75.00,79.86,72.93,16.0,12.0,17.0,17.0,16.0,16.0,8.0,16.0,8.5,8.5,12.5,12.5,1017.0,1008.0,1027.0,1028.0,81.0,81.0,89.0,89.0
2,5.0,5.0,10.0,5.0,9.0,9.0,5.0,5.0,0.0,0.3,0.2,0.1,1006.0,1006.0,1016.0,1016.0,58.0,71.0,63.0,58.0,11.71,10.67,13.57,12.29,11.57,13.00,6.29,9.36,4.70,7.97,5.61,3.63,1009.71,1007.33,1020.57,1022.14,71.71,77.33,78.43,72.00,16.0,15.0,17.0,17.0,16.0,16.0,11.0,20.0,19.6,19.6,12.5,12.5,1015.0,1009.0,1027.0,1028.0,88.0,88.0,89.0,89.0
3,5.0,12.0,9.0,5.0,8.0,8.0,5.0,5.0,0.0,0.5,0.2,0.1,1006.0,1007.0,1017.0,1012.0,58.0,71.0,63.0,58.0,11.43,13.33,13.14,12.14,11.29,11.00,6.86,10.57,4.77,8.03,5.27,3.40,1009.43,1009.67,1022.29,1021.86,74.29,80.33,75.14,70.07,16.0,15.0,17.0,17.0,16.0,16.0,11.0,22.0,19.6,19.6,12.5,12.5,1013.0,1013.0,1028.0,1028.0,88.0,88.0,89.0,89.0
4,5.0,12.0,5.0,5.0,5.0,5.0,5.0,5.0,0.3,0.5,0.2,0.1,1006.0,1009.0,1018.0,1009.0,58.0,82.0,63.0,58.0,10.86,13.33,11.71,11.64,10.57,7.33,7.86,11.50,5.39,8.13,5.23,3.49,1009.86,1012.67,1023.00,1021.29,77.29,86.00,72.86,69.07,15.0,15.0,17.0,17.0,16.0,9.0,12.0,22.0,19.6,19.6,12.5,12.5,1016.0,1016.0,1028.0,1028.0,88.0,88.0,89.0,89.0


Get all values for the same numerical variables in different time and do a mean function in order to have one indicatioe columns instead of 4 columns

In [104]:
index=0
incr_index = 4
counter = (Numerical_df.shape)[1]
while(index<counter):
  df_new = Numerical_df.iloc[:,index:index+incr_index]
  Numerical_df['ind_'+list(Numerical_df.columns)[index]] = df_new.mean(axis=1)
  index = index + 4

Numerical_df=Numerical_df.drop(Numerical_df.columns.to_series()['min_wind_speed_prev7d':'max_humidity_f_next14d'], axis=1)
Numerical_df

,ind_min_wind_speed_prev7d,ind_min_temperature_prev7d,ind_min_rain_mm_prev7d,ind_min_pressure_prev7d,ind_min_humidity_prev7d,ind_mean_wind_speed_prev7d,ind_mean_temperature_prev7d,ind_mean_rain_mm_prev7d,ind_mean_pressure_prev7d,ind_mean_humidity_prev7d,ind_max_wind_speed_prev7d,ind_max_temperature_prev7d,ind_max_rain_mm_prev7d,ind_max_pressure_prev7d,ind_max_humidity_prev7d
0,6.25,6.25,0.150,1007.50,56.00,11.5375,9.1075,3.9075,1014.4625,71.9175,15.50,12.25,14.050,1020.00,85.00
1,6.25,7.00,0.150,1009.50,60.75,11.3575,9.7675,3.5800,1014.5700,73.5550,15.50,14.00,10.500,1020.00,85.00
2,6.25,7.00,0.150,1011.00,62.50,12.0600,10.0550,5.4775,1014.9375,74.8675,16.25,15.75,16.050,1019.75,88.50
3,7.75,6.50,0.200,1010.50,62.50,12.5100,9.9300,5.3675,1015.8125,74.9575,16.25,16.25,16.050,1020.50,88.50
4,6.75,5.00,0.275,1010.50,65.25,11.8850,9.3150,5.5600,1016.7050,76.3050,16.00,14.75,16.050,1022.00,88.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198121,6.25,23.75,0.000,1010.00,39.50,7.8125,28.2475,0.6975,1014.4525,47.9400,10.00,31.50,5.425,1017.75,64.25
198122,6.00,24.25,0.000,1010.00,39.50,8.0300,28.3375,0.7200,1014.3625,47.9975,10.50,31.50,5.450,1017.50,64.75
198123,6.25,24.50,0.000,1010.75,39.50,7.9875,28.4225,0.4600,1014.7850,47.5725,10.50,31.50,3.700,1017.50,62.00
198124,6.25,24.50,0.000,1011.50,39.75,7.8825,28.6325,0.4400,1015.5300,47.2950,10.50,31.75,3.600,1018.50,60.50
